# Inference on finetuned Stable Diffusion 

In [ ]:
#!git clone https://github.com/huggingface/diffusers
#!pip install -r diffusers/examples/text_to_image/requirements.txt
#!pip install huggingface-hub

In [ ]:
!accelerate config default

In [ ]:
from huggingface_hub import login

# Login to Hugging Face
login()

## Generate Single Image

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
model_path = "finetuned-stable-diffusion"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="Generate an image with an an abnormality located in the center, upper-left, upper-right, lower-left, lower-right, center-left, center-right, upper-center and lower-center.").images[0]
image.save("gen-finetuned1.png")

## Generate Multiple Image

In [ ]:

import os
import csv
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

# Set up the pipeline
model_path = "finetuned-stable-diffusion"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

# Disable NSFW filter
pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))

# Define file paths
prompt_file_path = "ImageCLEFmed-MEDVQA-GI-2024-Testing-Propmpts.txt"
output_folder = "SD-synthetic"
csv_file_path = "prompt_and_data.csv"

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Open the prompt file and read prompts
with open(prompt_file_path, 'r') as file:
    prompts = [next(file).strip() for _ in range(50)]

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['prompt', 'image_name', 'image_path']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Generate images for each prompt and save them
    for i, prompt in enumerate(prompts):
        prompt = prompt.strip()
        if prompt:  # Check if prompt is not empty
            result = pipe(prompt=prompt)
            image = result.images[0]
            image_name = f"gen-finetuned-{i+1}.png"
            image_path = os.path.join(output_folder, image_name)
            image.save(image_path)

            # Write prompt and image details to the CSV file
            writer.writerow({'prompt': prompt, 'image_name': image_name, 'image_path': image_path})
